In [ ]:
from IPython.display import display, Markdown
import ipywidgets as ipw
import widgets
import utils
import os
import json

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

inventory_types = [
    ("Select inventory object type...", -1),
    ("Crystal", "Crystal"),
    ("Crystal Concept", "CrystalConcept"),
    ("Molecule", "Molecule"),
    ("Molecule Concept", "MoleculeConcept"),
    ("Reaction Product Concept", "ReactionProductConcept"),
    ("Reaction Product", "ReactionProduct"),
    ("2D Layer Material", "TwoDLayerMaterial")
] 

inventory_type_dropdown = utils.Dropdown(
    options = inventory_types,
    value = -1
)

object_widget = ipw.VBox()

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))
    
def select_inventory_type(change):
    inventory_type = inventory_type_dropdown.value
    if inventory_type == -1:
        object_widget.children = []
        return

    object_props_widgets = widgets.OpenbisObjectWidget(inventory_type, DATA_MODEL)
    
    if inventory_type in ["Crystal", "Molecule", "TwoDLayerMaterial"]:
        checkbox_on_change(
            object_props_widgets.properties_widgets["hazardous"]["value_widget"], 
            object_props_widgets.properties_widgets["hazardous_specification"]["value_widget"]
        )
        checkbox_on_change(
            object_props_widgets.properties_widgets["other_storage_condition"]["value_widget"], 
            object_props_widgets.properties_widgets["other_storage_condition_specification"]["value_widget"]
        )
    
    object_widget.children = [object_props_widgets]

def create_object_openbis(b):
    inventory_type = inventory_type_dropdown.value
    if inventory_type == -1:
        return

    if inventory_type == "Molecule":
        object_selector = object_widget.children[0].properties_widgets["molecule_concept"]["value_widget"].dropdown
        if object_selector.value == -1:
            display(utils.Javascript(data = "alert('Select a molecule concept.')"))
            return
        
        # Check if the molecule is already in openBIS
        molecules_empa_id_openbis = [f"{molecule.props['empa_number']}-{molecule.props['batch']}" for molecule in utils.get_openbis_objects(OPENBIS_SESSION, type ="MOLECULE")]
        object_properties = object_widget.children[0].get_values()
        
        if "empa_number" not in object_properties:
            display(utils.Javascript(data = "alert('Empa number was not specified.')"))
            return
        else:
            object_properties["empa_number"] = int(object_properties["empa_number"])

        if "batch" not in object_properties:
            display(utils.Javascript(data = "alert('Batch was not specified.')"))
            return
        
        molecule_empa_id = f"{object_properties['empa_number']}-{object_properties['batch']}"
        if molecule_empa_id in molecules_empa_id_openbis:
            display(utils.Javascript(data = "alert('Molecule is already in openBIS!')"))
            return
        
        molecule_object = utils.create_openbis_object(
            OPENBIS_SESSION, type="MOLECULE", 
            collection="/MATERIALS/MOLECULES/PRECURSOR_COLLECTION", 
            props = object_properties
        )
        
    # utils.upload_datasets(OPENBIS_SESSION, molecule_object, molecule_support_files, "RAW_DATA")
    display(utils.Javascript(data = "alert('Upload successful!')"))

def checkbox_on_change(checkbox, textbox):
    textbox.disabled = True
    def enable_textbox(change):
        if checkbox.value:
            textbox.disabled = False
        else:
            textbox.disabled = True
    checkbox.observe(enable_textbox, names = "value")

In [ ]:
inventory_type_dropdown.observe(select_inventory_type, names = 'value')

# Register/edit inventory objects

## Select object type

In [ ]:
display(inventory_type_dropdown)

## Properties

In [ ]:
display(object_widget)

## Support files

In [ ]:
# display(molecule_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_object_openbis)
quit_button.on_click(close_notebook)